In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_csv('faers/data/raw/2006/DRUG06Q1.TXT', sep='$')
print(df.head())

       ISR    DRUG_SEQ ROLE_COD            DRUGNAME  VAL_VBM ROUTE  \
0  4793437  1006499909       PS       GLUCOPHAGE XR        1  ORAL   
1  4793437  1006499910       SS                EDTA        2   NaN   
2  4793437  1006499911        C           AVANDAMET        1   NaN   
3  4799889  1006520185       PS             EFFEXOR        1  ORAL   
4  4799889  1006533234       SS  ETHANOL (ETHANOL,)        2  ORAL   

                DOSE_VBM DECHAL RECHAL LOT_NUM  EXP_DT  NDA_NUM  VAR13  
0                    NaN    NaN    NaN     NaN     NaN      NaN    NaN  
1                    NaN    NaN    NaN     NaN     NaN      NaN    NaN  
2                    NaN    NaN    NaN     NaN     NaN      NaN    NaN  
3                   ORAL      D      D     NaN     NaN  20151.0    NaN  
4  1 BOTTLE OF PORT ORAL      D      D     NaN     NaN      NaN    NaN  


In [3]:
drugname_df = df[df['DRUGNAME'].str.contains(r'[A-Za-z0-9]', na=False)]
drugname_df['DRUGNAME'] = drugname_df['DRUGNAME'].fillna('').astype('string')
drugname_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 336291 entries, 0 to 336518
Data columns (total 13 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   ISR       336291 non-null  int64  
 1   DRUG_SEQ  336291 non-null  int64  
 2   ROLE_COD  336291 non-null  object 
 3   DRUGNAME  336291 non-null  string 
 4   VAL_VBM   336291 non-null  int64  
 5   ROUTE     174687 non-null  object 
 6   DOSE_VBM  77393 non-null   object 
 7   DECHAL    56671 non-null   object 
 8   RECHAL    55715 non-null   object 
 9   LOT_NUM   16705 non-null   object 
 10  EXP_DT    4495 non-null    float64
 11  NDA_NUM   85534 non-null   float64
 12  VAR13     0 non-null       float64
dtypes: float64(3), int64(3), object(6), string(1)
memory usage: 35.9+ MB


/var/folders/cq/5z83380x2rj6r_ldbd5l4r100000gn/T/ipykernel_27910/3301206248.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drugname_df['DRUGNAME'] = drugname_df['DRUGNAME'].fillna('').astype('string')


In [4]:
drugname_df['DRUGNAME'] = drugname_df['DRUGNAME'].str.strip().str.lower()

# Define a pattern that matches all common quote characters
quote_chars = r'["“”‘’\'`]'

# Remove all types of quotes anywhere in the string
drugname_df['DRUGNAME'] = drugname_df['DRUGNAME'].str.replace(quote_chars, '', regex=True)

# Strip whitespace again just in case
drugname_df['DRUGNAME'] = drugname_df['DRUGNAME'].str.strip()

# Sort
drugname_df = drugname_df.sort_values(by='DRUGNAME').reset_index(drop=True)

/var/folders/cq/5z83380x2rj6r_ldbd5l4r100000gn/T/ipykernel_27910/3732365550.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drugname_df['DRUGNAME'] = drugname_df['DRUGNAME'].str.strip().str.lower()
/var/folders/cq/5z83380x2rj6r_ldbd5l4r100000gn/T/ipykernel_27910/3732365550.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drugname_df['DRUGNAME'] = drugname_df['DRUGNAME'].str.replace(quote_chars, '', regex=True)
/var/folders/cq/5z83380x2rj6r_ldbd5l4r100000gn/T/ipykernel_27910/3732365550.py:10: SettingW

In [5]:
removed_df = drugname_df['DRUGNAME'].str.replace(r'[^A-Za-z0-9() -]', '', regex=True)
removed_df.to_csv('removed_drugnames.csv', columns=['DRUGNAME'], index=False)
print(removed_df.info())

<class 'pandas.core.series.Series'>
RangeIndex: 336291 entries, 0 to 336290
Series name: DRUGNAME
Non-Null Count   Dtype 
--------------   ----- 
336291 non-null  string
dtypes: string(1)
memory usage: 2.6 MB
None


In [6]:
# Select rows where DRUGNAME contains at least one non-alphanumeric character
non_alnum_df = drugname_df[drugname_df['DRUGNAME'].str.contains(r'[^A-Za-z ]', na=False)]
print(len(non_alnum_df))
non_alnum_df.to_csv('nonalnum_drugnames.csv', columns=['DRUGNAME'], index=False)

38552


In [7]:
# Get sorted unique drug names
removed_df = removed_df.str.replace(r'\(.*', '', regex=True).str.strip()
removed_df = removed_df.str.replace(r' .+', '', regex=True).str.strip()


drug_counts = removed_df.value_counts().reset_index()
drug_counts.columns = ['DRUGNAME', 'COUNT']
drug_counts.to_csv('drugname_counts.csv')



In [8]:
# drug_counts = drug_counts[drug_counts['COUNT'] > 10]
drug_counts = drug_counts[drug_counts['DRUGNAME'].str.contains(r'[A-Za-z]', na=False)]
drugname_df = drugname_df[drugname_df['DRUGNAME'].isin(drug_counts['DRUGNAME'])]

In [9]:
drugname_df.to_csv('drugnames.csv', columns=['DRUGNAME'], index=False)